1. 문서의 내용을 읽고
2. 문서를 쪼갠다
- 토큰 수 초과로 답변을 생성하지 못하거나
- 문서가 길면 (인풋) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때 벡터 DB 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달하는 식으로 구현

In [ ]:
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [3]:
import os
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

c:\Users\ryumo\projects\AI-langchain-RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = "연봉 5천만원의 직장인의 소득세는 얼마인가요?"

In [13]:
retriever = database.as_retriever(search_kwargs={'k': 10})
retriever.invoke(query)

[Document(id='fe10f821-8eaa-48f9-b60c-2f11206075a7', metadata={'source': './tax_with_markdown.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1

In [14]:
document_list[52]

Document(metadata={'source': './tax_with_markdown.docx', 'text': '제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (10억원을 초과하는 금액의 45퍼센트)|\n\n\n\n\n\n② 거주자의 퇴직소득에 대한 소득세는 다음 각 호의 순서에 따라 계산한 금액(이하 “퇴직소득 산출세액”이라 한다)으로 한

In [15]:
from langchain_openai import ChatOpenAI
from langsmith import Client

llm = ChatOpenAI(model="gpt-4o")

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langsmith import Client

# LangSmith에서 프롬프트 불러오기
client = Client()
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

# ✅ retriever 정의 (database → vectordb 또는 chroma_client로 바꿔야 할 수도 있음)
retriever = database.as_retriever()

# ✅ format_docs 함수 누락 시 아래 추가
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# ✅ LLM 객체 반드시 선언되어 있어야 함
# 예: from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-4o-mini")

# ✅ RAG 체인 구성
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# ✅ 실행
query = "연봉 5천만원의 직장인의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke(query)
print(ai_message)

정확한 소득세 금액을 계산하려면 기준세율과 소득공제, 세액공제 등을 고려해야 합니다. 제공된 문서에서는 근로소득에 기본세율이 적용된다고만 언급되어 있고, 구체적인 연봉 5천만 원에 대한 소득세 계산 방법은 제공되지 않았습니다. 따라서 정확한 소득세 금액을 계산하기 위해서는 세무 전문가나 소득세 계산기를 사용하는 것이 가장 좋습니다.


소득세는 기본세율을 적용받는 근로소득의 경우로 설명됩니다. 연봉 5천만 원에 대한 정확한 소득세를 계산하려면 공제 항목 등을 고려해야 하므로, 기본세율만으로는 정확한 금액을 제공할 수 없습니다. 세율과 공제 사항은 개인의 상황에 따라 다르기 때문에, 연말정산 등을 통해 정확히 계산해야 합니다.

In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langsmith import Client
from langchain_openai import ChatOpenAI

# ✅ LangSmith에서 프롬프트 불러오기
client = Client()
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

# ✅ LLM 설정
llm = ChatOpenAI(model="gpt-4o")

# ✅ retriever 정의
retriever = database.as_retriever()

# ✅ format_docs 함수 정의
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# ✅ RAG 체인 구성
qa_chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# ✅ 실행
query = "연봉 5천만원의 직장인의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke(query)
print(ai_message)


제가 제공받은 정보로는 연봉 5천만 원의 직장인의 소득세를 정확히 계산할 수 없습니다. 근로소득에 대한 세율이 기본 세율이라는 정보가 주어졌으나, 구체적인 세율 정보가 필요합니다. 기본 세율은 종합소득세에 따라 정해지므로, 필요한 세율 정보를 추가로 확인해야 합니다.


In [18]:
print(prompt.pretty_repr())


================================ Human Message =================================

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [16]:
retriever = database.as_retriever(search_kwargs={"k": 10})

qa_chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough()
    }
    | ChatPromptTemplate.from_template("""
    당신은 세무사입니다. 
    아래의 문서 내용을 바탕으로만 답변하세요.
    
    문서 내용:
    {context}

    질문:
    {question}
    """)
    | llm
    | StrOutputParser()
)

query = "연봉 5천만원의 직장인의 소득세는 얼마인가요?"
ai_message = qa_chain.invoke(query)
print(ai_message)


연봉 5천만원의 직장인의 소득세를 계산하기 위해서는 종합소득과세표준과 해당 세율을 적용해야 합니다. 문서에 나온 종합소득 과세표준에 따른 세율은 다음과 같습니다:

- 1,400만원 이하: 과세표준의 6%
- 1,400만원 초과 5,000만원 이하: 84만원 + (1,400만원을 초과하는 금액의 15%)

연봉 5천만원의 경우, 과세표준을 적용하면 다음과 같이 계산할 수 있습니다.

1. 1,400만원까지의 세금:
   - 1,400만원 x 6% = 84만원

2. 1,400만원을 초과한 3,600만원(5,000만원 - 1,400만원)에 대한 세금:
   - 3,600만원 x 15% = 540만원

3. 총 소득세:
   - 84만원 + 540만원 = 624만원

따라서, 연봉 5천만원의 직장인은 624만원의 소득세를 납부해야 합니다.


In [20]:
retriever.invoke(query)

[Document(id='fe10f821-8eaa-48f9-b60c-2f11206075a7', metadata={'source': './tax_with_markdown.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1

In [47]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]
dictionary_str = "\n".join(dictionary) # 리스트를 문자열로 변환

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary_str}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [49]:
# 1. 질문 변경 체인 실행 (에러 해결)
new_question = dictionary_chain.invoke({"question": query})
print(f"새로운 질문: {new_question}")

# 2. RAG 체인 실행 (변경된 질문 사용)
ai_response = qa_chain.invoke(new_question) 
print(ai_response)

새로운 질문: 연봉 5천만원의 거주자의 소득세는 얼마인가요?
연봉 5천만원의 거주자의 소득세를 계산하기 위해서는 제55조에 명시된 종합소득 과세표준 구간별 세율을 적용해야 합니다. 

1. 연봉 5천만원 이하 구간에 대해 조세표준이 1,400만원을 초과하고 5,000만원 이하이므로 해당 구간의 세율을 적용해야 합니다. 

2. 1,400만원 초과 ~ 5,000만원 이하 구간의 세율은 기본 세액 84만원에 1,400만원을 초과하는 금액의 15%를 추가로 부과합니다.

따라서, 5,000만원 연봉의 과세 구간은 다음과 같이 계산됩니다:

- 기본 세액: 84만원
- 추가 세액: (5천만원 - 1,400만원) * 0.15 = 3,600만원 * 0.15 = 540만원

총 소득세 = 84만원 + 540만원 = 624만원

따라서 연봉 5천만원의 거주자의 소득세는 총 624만원입니다.


In [50]:
query

'연봉 5천만원의 직장인의 소득세는 얼마인가요?'